# Парсим мемы в питоне: как обойти серверную блокировку.


# 1. Вламываемся в мемохранилище

## 1.1. Что мы хотим получить

Итак, мы хотим распарсить [knowyourmeme.com](http://knowyourmeme.com) и получить кучу разных переменных:

- **Name** – название мема,
- **Origin_year** – год его создания,
- **Views** – число просмотров,
- **About** – текстовое описание мема,
- ** и многие другие**

Более того, мы хотим сделать это без вот этого всего:



После скачивания и чистки данных от мусора, можно будет заняться строительством моделей. Например, попытаться предсказать популярность мема по его параметрам. Но это все позже, а сейчас познакомимся с парой определений:

* **Парсер** — это скрипт, который грабит информацию с сайта
* **Краулер** — это часть парсера, которая бродит по ссылкам
* **Краулинг** — это переход по страницам и ссылкам
* **Скрапинг** — это сбор данных со страниц
* **Парсинг** — это сразу и краулинг и скрапинг!


## 1.2.  Что такое HTML

**HTML (HyperText Markup Language)**  — это такой же язык разметки как Markdown или LaTeX. Он является стандартным для написания различных сайтов. Команды в таком языке называются **тегами**. Если открыть абсолютно любой сайт, нажать на правую кнопку мышки, а после нажать `View page source`, то перед вами предстанет HTML скелет этого сайта.

Можно увидеть, что HTML-страница это ни что иное как набор вложенных тегов. Можно заметить, например, следующие теги:

- `<title>` – заголовок страницы
- `<h1>…<h6>` – заголовки разных уровней
- `<p>` – абзац (paragraph)
- `<div>` – выделения фрагмента документа с целью изменения вида содержимого
- `<table>` – прорисовка таблицы
- `<tr>` – разделитель для строк в таблице
- `<td>` – разделитель для столбцов в таблице
- `<b>` – устанавливает жирное начертание шрифта

Обычно команда `<...>` открывает тег, а  `</...>` закрывает его. Все, что находится между этими двумя командами, подчиняется правилу, которое диктует тег. Например, все, что находится между `<p>` и  `</p>` — это отдельный абзац.   

Теги образуют своеобразное дерево с корнем в теге `<html>` и разбивают страницу на разные логические кусочки. У каждого тега могут быть свои потомки (дети) — те теги, которые вложены в него, и свои родители.

Например, HTML-древо страницы может выглядеть вот так:

    <html>
    <head> Заголовок </head>
    <body>
        <div>
            Первый кусок текста со своими свойствами
        </div>
        <div>
            Второй кусок текста
                <b>
                    Третий, жирный кусок
                </b>
        </div>
        Четвёртый кусок текста        
    </body>
    </html>            
    
    


Можно работать с этим html как с текстом, а можно как с деревом. Обход этого дерева и есть парсинг веб-страницы. Мы всего лишь будем находить нужные нам узлы среди всего этого разнообразия и забирать из них информацию!

Вручную обходить эти деревья не очень приятно, поэтому есть специальные языки для обхода деревьев.



## 1.3. Наш первый запрос

Доступ к веб-станицам позволяет получать модуль `requests`. Подгрузим его. За компанию подгрузим ещё парочку дельных пакетов.

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [7]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для времени

Для наших благородных исследовательских целей нужно собрать данные по каждому мему с соответствующей ему страницы. Но для начала нужно получить адреса этих страниц. Поэтому открываем основную страницу со всеми выложенными мемами. Выглядит она следующим образом:



Со страницы всех мемов http://knowyourmeme.com/memes/all мы  будем тащить ссылки на каждый из перечисленных мемов. Сохраним в переменную `page_link` адрес основной страницы и откроем её при помощи библиотеки `requests`.

In [1]:
page_link = 'https://www.vprok.ru/promos/11154/promocode-new700?filter.strana=rossiya'

Не будем показывать серверу, что мы сидим на питоне и используем библиотеку requests. Это может вызвать у него некоторые подозрения относительно наших благих намерений и он решит нас безжалостно отвергнуть.


Очевидно, что нашему скромному запросу не тягаться с таким обилием мета-информации, которое передается при запросе из обычного браузера. К счастью, никто нам не мешает притвориться человечными и пустить пыль в глаза сервера при помощи генерации фейкового юзер-агента. Библиотек, которые справляются с такой задачей, существует очень и очень много, лично мне больше всего нравится [`fake-useragent`](https://pypi.python.org/pypi/fake-useragent). При вызове метода из различных кусочков будет генерироваться рандомное сочетание операционной системы, спецификаций и версии браузера, которые можно передавать в запрос:

In [4]:
!pip install fake_useragent

In [3]:
# подгрузим один из методов этой библиотеки
from fake_useragent import UserAgent

In [4]:
UserAgent().chrome

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'

In [5]:
UserAgent().safari

'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.6 Safari/605.1.15'

In [8]:
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

<Response [200]>

In [9]:
type(response)

requests.models.Response

Замечательно, наша небольшая маскировка сработала и обманутый сервер покорно выдал благословенный 200 ответ — соединение установлено и данные получены, всё чудесно! Посмотрим, что же все-таки мы получили.

In [10]:
html = response.content

In [11]:
html[:1000]

b"<!DOCTYPE html>\n<html xmlns:fb='https://www.facebook.com/2008/fbml' xmlns='https://www.w3.org/1999/xhtml'>\n<head>\n<meta content='width=1060' name='viewport'>\n<meta content='text/html; charset=utf-8' http-equiv='Content-Type'>\n\n<meta property='og:title' content='All Entries' />\n<meta property='og:site_name' content='Know Your Meme' />\n<meta property='og:image' content='https://a.kym-cdn.com/assets/kym-logo-large-2be3f3818691470a0369e154647ca0f0.png' />\n<meta property='og:type' content='article' />\n<meta property='fb:app_id' content='104675392961482' />\n<meta property='fb:pages' content='88519108736' />\n<meta property='article:publisher' content='https://www.facebook.com/knowyourmeme' />\n<meta name='twitter:card' content='summary_large_image' />\n<meta name='twitter:site' content='@knowyourmeme' />\n<meta name='twitter:creator' content='@knowyourmeme' />\n<meta name='twitter:title' content='All Entries' />\n<meta name='twitter:description' content='Know Your Meme | Our con

In [12]:
html[-1000:]

b'ef="https://knowyourmeme.com/terms-of-service" rel="nofollow" target="_blank">terms of use</a> and <a href="https://knowyourmeme.com/privacy" rel="nofollow" target="_blank">privacy policy</a> and <a href="https://knowyourmeme.com/dmca" rel="nofollow" target="_blank">DMCA policy</a>. <br> &copy; 2007-2024 <a href="https://literally.media" target="_blank">Literally Media Ltd.</a> </p> </div> </div>\n<div class=\'hide\'> <div id=\'login_signup_form\'> <h3>Yo Yo! You must login or signup first!</h3> <table> <tr> <td class=\'login\'> <h6>Already a memeber?</h6> <p><a href="https://knowyourmeme.com/login" class="large red button" rel="nofollow">Login Now!</a></p> </td> <td class=\'signup\'> <h6>Don\'t have an account?</h6> <p><a href="https://knowyourmeme.com/signup" class="large red button" rel="nofollow">Sign up Now!</a></p> </td> </tr> </table> </div> </div> <div class=\'hide\' data-sections=\'Entry Galleries\' id=\'chartbeat-sections\'></div>\n<div data-group="control" id="split-test" 

In [13]:
len(html)

46434

In [14]:
type(html)

bytes

Выглядит неудобоваримо, как насчет сварить из этого дела что-то покрасивее? Например, красивый суп.

## 1.4. Красивый суп



Пакет **[bs4 , a.k.a BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)** (тут есть гиперссылка на лучшего друга человека — документацию) был назван в честь стишка про красивый суп из Алисы в стране чудес.

Красивый суп — это совершенно волшебная библиотека, которая из сырого и необработанного HTML кода страницы выдаст вам структурированный массив данных, по которому очень удобно искать необходимые теги, классы, атрибуты, тексты и прочие элементы веб страниц.

> Пакет под названием `BeautifulSoup` — скорее всего, не то, что нам нужно. Это третья версия (*Beautiful Soup 3*), а мы будем использовать четвертую. Нужно будет установить пакет `beautifulsoup4`. Чтобы было совсем весело, при импорте нужно указывать другое название пакета — `bs4`, а импортировать функцию под названием `BeautifulSoup`. В общем, сначала легко запутаться, но эти трудности нужно преодолеть.

С необработанным XML кодом страницы пакет также работает (XML — это исковерканый и превращённый в диалект, с помощью своих команд, HTML). Для того, чтобы пакет корректно работал с XML разметкой, придётся в довесок ко всему нашему арсеналу установить пакет `xml`.

In [15]:
from bs4 import BeautifulSoup

Передадим функции `BeautifulSoup` текст веб-страницы, которую мы недавно получили.

In [16]:
soup = BeautifulSoup(html, 'html.parser') # В опции также можно указать lxml,
                                         # если предварительно установить одноименный пакет

In [17]:
type(soup)

bs4.BeautifulSoup

In [18]:
print(soup.prettify()[:2000])

<!DOCTYPE html>
<html xmlns="https://www.w3.org/1999/xhtml" xmlns:fb="https://www.facebook.com/2008/fbml">
 <head>
  <meta content="width=1060" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="All Entries" property="og:title">
   <meta content="Know Your Meme" property="og:site_name">
    <meta content="https://a.kym-cdn.com/assets/kym-logo-large-2be3f3818691470a0369e154647ca0f0.png" property="og:image"/>
    <meta content="article" property="og:type"/>
    <meta content="104675392961482" property="fb:app_id"/>
    <meta content="88519108736" property="fb:pages"/>
    <meta content="https://www.facebook.com/knowyourmeme" property="article:publisher"/>
    <meta content="summary_large_image" name="twitter:card"/>
    <meta content="@knowyourmeme" name="twitter:site"/>
    <meta content="@knowyourmeme" name="twitter:creator"/>
    <meta content="All Entries" name="twitter:title"/>
    <meta content="Know Your Meme | Our confirmed s

Получим что-то вот такое:
    
```
<!DOCTYPE html>

<html xmlns="http://www.w3.org/1999/xhtml" xmlns:fb="http://www.facebook.com/2008/fbml">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"c1a6d52f38","applicationID":"31165848","transactionName":"dFdfRUpeWglTQB8GDUNKWFRLHkUNWUU=","queueTime":0,"applicationTime":24,"agent":""}</script>
<script type="text/javascript">window.NREUM||(NREUM={}),__nr_require=function(e,n,t){function r(t){if(!n[t]){var o=n[t]={exports:{}};e[t][0].call(o.exports,function(n){var o=e[t][1][n];return r(o||n)},o,o.exports)}return n[t].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<t.length;o++)r(t[o]);return r}({1:[function(e,n,t){function r(){}function o(e,n,t){return function(){return i(e,[c.now()].concat(u(arguments)),n?null:this,t),n?void 0:this}}var i=e("handle"),a=e(2),u=e(3),f=e("ee").get("tracer"),c=e("loader"),s=NREUM;"undefined"==typeof window.newrelic&&(newrelic=s);var p=
```

Стало намного лучше, не правда ли? Что же лежит в переменной `soup`? Невнимательный пользователь, скорее всего, скажет,что ничего вообще не изменилось. Тем не менее, это не так. Теперь мы можем свободно бродить по HTML-дереву страницы, искать детей, родителей и вытаскивать их!

Например, можно бродить по вершинам, указывая путь из тегов.

In [19]:
soup.html.head.title

<title>All Entries | Know Your Meme</title>

In [20]:
type(soup.html.head.title)

bs4.element.Tag

Можно вытащить из того места, куда мы забрели, текст с помощью метода `text`.

In [21]:
soup.html.head.title.text

'All Entries | Know Your Meme'

In [22]:
type(soup.html.head.title.text)

str

Более того, зная адрес элемента, мы сразу можем найти его. Например, можно сделать это по классу. Следующая команда должна найти элемент, который лежит внутри тега `a` и имеет класс `photo`.

In [23]:
obj = soup.find('a', attrs = {'class':'photo'})
obj

<a class="photo left" href="https://knowyourmeme.com/memes/events/mizkif-assaulting-mitch-jones-allegations" target="_self"><img alt="Mitch Jones Uses Video Footage To Allege Fellow Streamer Mizkif Assaulted Him And Inflicted Serious Injuries" fetchpriority="low" height="112" src="https://i.kym-cdn.com/featured_items/icons/wide/000/022/918/1.jpg" title="Mitch Jones Uses Video Footage To Allege Fellow Streamer Mizkif Assaulted Him And Inflicted Serious Injuries" width="198"/> <div class="info abs"> <div class="c"> Mitch Jones Uses Video Footage To Allege Fellow Streamer Mizkif Assaulted Him And Inflicted Serious Injuries </div> </div> </a>

In [24]:
obj.get("class")

['photo', 'left']

Однако, вопреки нашим ожиданиям, вытащенный объект имеет класс `"photo left"`. Оказывается, `BeautifulSoup4` расценивает аттрибуты `class` как набор отдельных значений, поэтому `"photo left"` для библиотеки равносильно `["photo", "left"]`, а указанное нами значение этого класса `"photo"` входит в этот список. Чтобы избежать такой неприятной ситуации и проходов по ненужным нам ссылкам, придется воспользоваться собственной функцией и задать строгое соответствие:

In [25]:
'photo' in ['photo', 'left']

True

In [26]:
obj = soup.find(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
obj

<a class="photo" href="https://knowyourmeme.com/memes/people/bianca-censori"><img alt="Kanye West and Bianca Censori" data-src="https://i.kym-cdn.com/entries/icons/medium/000/048/347/Thumbnail_template_-_2024-02-09T055451.272.jpg" loading="lazy" src="https://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Bianca Censori"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #d32f2e; color: white;">Person</span> </div> </a>

In [27]:
obj.get("class")

['photo']

In [28]:
obj

<a class="photo" href="https://knowyourmeme.com/memes/people/bianca-censori"><img alt="Kanye West and Bianca Censori" data-src="https://i.kym-cdn.com/entries/icons/medium/000/048/347/Thumbnail_template_-_2024-02-09T055451.272.jpg" loading="lazy" src="https://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Bianca Censori"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #d32f2e; color: white;">Person</span> </div> </a>

Полученный после поиска объект также обладает структурой bs4. Поэтому можно продолжить искать нужные нам объекты уже в нём! Вытащим ссылку на этот мем. Сделать это можно по атрибуту `href`, в котором лежит наша ссылка.

In [29]:
obj.attrs['href']

'https://knowyourmeme.com/memes/people/bianca-censori'

Обратите внимание, что после всех этих безумных преобразований у данных поменялся тип. Теперь они `str`. Это означет, что с ними можно работать как с текстом и пускать в ход для отсеивания лишней информации регулярные выражения.

In [30]:
print("Тип данных до вытаскивания ссылки:", type(obj))
print("Тип данных после вытаскивания ссылки:", type(obj.attrs['href']))

Тип данных до вытаскивания ссылки: <class 'bs4.element.Tag'>
Тип данных после вытаскивания ссылки: <class 'str'>


Если несколько элементов на странице обладают указанным адресом, то метод `find` вернёт только самый первый.  Чтобы найти все элементы с таким адресом, нужно использовать метод `findAll`, и на выход будет выдан список. Таким образом, мы можем получить одним поиском сразу все объекты, содержащие ссылки на страницы с мемами.

In [31]:
meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
meme_links[:3]

[<a class="photo" href="https://knowyourmeme.com/memes/people/bianca-censori"><img alt="Kanye West and Bianca Censori" data-src="https://i.kym-cdn.com/entries/icons/medium/000/048/347/Thumbnail_template_-_2024-02-09T055451.272.jpg" loading="lazy" src="https://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Bianca Censori"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> <span class="label" style="background: #d32f2e; color: white;">Person</span> </div> </a>,
 <a class="photo" href="https://knowyourmeme.com/memes/taylor-swift-jetting-everywhere"><img alt="Taylor Swift Jetting Everywhere" data-src="https://i.kym-cdn.com/entries/icons/medium/000/048/346/cover15.jpg" loading="lazy" src="https://a.kym-cdn.com/assets/blank-b3f96f160b75b1b49b426754ba188fe8.gif" title="Taylor Swift Jetting Everywhere"/> <div class="entry-labels"> <span class="label label-submission"> Submission </span> </div> </a>,
 <a class="photo" href="https://kno

In [32]:
len(meme_links)

16

Осталось очистить полученный список от мусора:

In [33]:
meme_links = [link.attrs['href'] for link in meme_links]

In [34]:
meme_links[:10]

['https://knowyourmeme.com/memes/people/bianca-censori',
 'https://knowyourmeme.com/memes/taylor-swift-jetting-everywhere',
 'https://knowyourmeme.com/memes/people-getting-arrested-over-big-boots',
 'https://knowyourmeme.com/memes/hey-are-you-listening-yeh',
 'https://knowyourmeme.com/memes/events/nikita-draguns-arrest',
 'https://knowyourmeme.com/memes/events/funimation-shutdown',
 'https://knowyourmeme.com/memes/draw-the-squad-playing-monopoly',
 'https://knowyourmeme.com/memes/what-a-nerd-we-always-loved-this',
 'https://knowyourmeme.com/memes/sad-baby-monkey',
 'https://knowyourmeme.com/memes/tekken-8-kazuya-finale-parodies']

In [35]:
len(meme_links)

16

Готово, получили чуть менее 20 по числу мемов на одной странице поиска.


Остался последний момент. Когда мы скачаем все мемы с текущей страницы, нам нужно будет каким-то образом забраться на соседнюю. На сайте это можно делать просто пролистывая страницу с мемами вниз, javascript-функции подтянут новые мемы на текущее окно, но сейчас трогать эти функции не хочется.

Обычно, все параметры, которые мы устанавливаем на сайте для поиска, отображаются на структуре хрефа. Мемы не являются исключением. Если мы хотим получить первую порцию мемов, мы должны будем обратиться к сайту по ссылке

                `http://knowyourmeme.com/memes/all/page/1`


Если мы захотим получить вторую поцию с шестнадцатью мемами, нам придётся немного видоизменить ссылку, а именно заменить номер страницы на 2.


                `http://knowyourmeme.com/memes/all/page/2`

Таким незатейливым образом мы сможем пройтись по всем страницам и ограбить мемохранилище.

Наконец, обернем в красивую функцию все-все манипуляции, проделанные выше:

In [36]:
def getPageLinks(page_number):
    """
        Возвращает список ссылок на мемы, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга

    """
    # составляем ссылку на страницу поиска
    page_link = 'http://knowyourmeme.com/memes/all/page/{}'.format(page_number)

    # запрашиваем данные по ней
    response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем пустой лист для текущей страницы
        return []

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    meme_links = soup.findAll(lambda tag: tag.name == 'a' and tag.get('class') == ['photo'])
    meme_links = ['http://knowyourmeme.com' + link.attrs['href'] for link in meme_links]

    return meme_links

Протестируем функцию и убедимся, что всё хорошо

In [37]:
meme_links = getPageLinks(1)
meme_links[:2]

['http://knowyourmeme.comhttps://knowyourmeme.com/memes/people/bianca-censori',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/taylor-swift-jetting-everywhere']

In [38]:
meme_links = getPageLinks(2)
meme_links[:2]

['http://knowyourmeme.comhttps://knowyourmeme.com/memes/events/counter-strike-2-gun-sticker-controversy',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/bad-word-bathroom-challenge']

Отлично, функция работает и теперь мы теоретически можем достать ссылки на все мемы, для чего нам придется пройтись по 2000+ страниц. Прежде чем расстраивать сервер таким количеством запросов, посмотрим, как доставать всю необходимую информацию о конкретном меме.

## 1.5 Финальная подготовка к грабежу

По аналогии со ссылками можно вытащить что угодно. Для этого надо сделать несколько шагов:

1. Открываем страничку с мемом
2. Находим любым способом тег для нужной нам информации
3. Пихаем всё это в красивый суп
4. ......
5. Profit

Для закрепления информации в голове любознательного читателя, вытащим число просмотров мема.

А в качестве примера возьмем самый популярный на этом сайте мем - Doge, набравший более 14 миллионов просмотров к 2023 году.

Сама страница, с которой мы будем доставать дорогую нашему исследовательскому сердцу информацию выглядит следуюшим образом:



Как и прежде, для начала сохраним ссылку на страницу в переменную и вытащим по ней контент.

In [39]:
meme_page = 'http://knowyourmeme.com/memes/doge'
meme_page

'http://knowyourmeme.com/memes/doge'

In [40]:
meme_page = 'http://knowyourmeme.com/memes/doge'

response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

html = response.content
soup = BeautifulSoup(html,'html.parser')

Посмотрим, как можно вытащить статистику просмотров, комментариев, а также числа загруженных видео и фото, связанных с нашим мемов. Всё это добро хранится справа вверху под тэгами `"dd"` и с классами  `"views"`, `"videos"`, `"photos"` и `"comments"`

In [41]:
views = soup.find(attrs={'class':'views'})
views

<dd class="views" title="14,121,050 Views">
<a href="/memes/doge" rel="nofollow">14,121,050</a>
</dd>

In [42]:
type(views)

bs4.element.Tag

In [43]:
views = views.get("title")
type(views)

str

In [44]:
views

'14,121,050 Views'

In [45]:
views.split()

['14,121,050', 'Views']

In [46]:
views = views.split()[0]
views

'14,121,050'

In [47]:
views = int(views.replace(',', ''))
views

14121050

Снова запихнём всё это в небольшую функцию.

In [48]:
def getStats(soup, stats):
    """
        Возвращает очищенное число просмотров/коментариев/...

        soup: объект bs4.BeautifulSoup
            суп текущей страницы

        stats: string
            views/videos/photos/comments

    """
    try:
        obj = soup.find(attrs={'class':stats})
        obj = obj.get("title")
        obj = obj.split()[0]
        obj = int(obj.replace(',', ''))
    except:
        obj=None

    return obj

Всё готово!

In [49]:
views = getStats(soup, stats='views')
videos = getStats(soup, stats='videos')
photos = getStats(soup, stats='photos')
comments = getStats(soup, stats='comments')

print("Просмотры: {}\nВидео: {}\nФото: {}\nКомментарии: {}".format(views, videos, photos, comments))

Просмотры: 14121050
Видео: 105
Фото: 1795
Комментарии: 923


Еще из интересного и исследовательского —  достанем дату и время добавления мема. Если посмотреть на страницу в браузере, можно подумать, что максимум информации, который мы можем вытащить - это число лет, прошедших с момента публикации —  `Added 9 years ago by NovaXP`. Однако мы так просто сдаваться не будем, полезем в кишки html и откопаем там кусок, ответственный за эту надпись:



Ага! Вот и подробности по дате добавления, с точностью до минуты. Элементарно

In [50]:
date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']
date

'2013-07-24T16:29:55-04:00'

На самом деле, парсеры — дело непредсказуемое. Часто страницы, которые мы парсим, имеют очень неоднородну структуру. Например, если мы парсим мемы, на части страниц может быть указано описание, а на части нет. Как только код впервые натыкается на отсутствие описания, он выдаёт ошибку и останавливается. Чтобы нормально собрать все данные, приходится прописывать исключения. Вроде бы, хранилище мемов хорошо оборудовано и никаких внештатных ситуаций происходить не должно. Тем не менее, очень не хочется проснуться утром и увидеть, что код сделал 20 итераций, нарвался на ошибку и отрубился.  Чтобы такого не произошло, можно, например, использовать конструкцию `try - except` и просто обрабатывать неугодные нам ошибки. Про исключения можно почитать [на просторах интернета](https://pythonworld.ru/tipy-dannyx-v-python/isklyucheniya-v-python-konstrukciya-try-except-dlya-obrabotki-isklyuchenij.html). В нашем же случае до ошибки можно и не доводить, а предварительно проверять, есть ли необходимый элемент на странице или нет при помощи обычного `if - else`, и уже после этого пытаться его распарсить.



## Truthy & Falsy python
[Подробнее](https://www.freecodecamp.org/news/truthy-and-falsy-values-in-python/)

In [51]:
a = None
if a:
    print(1)
else:
    print(2)

2


In [52]:
if [1, 2, 3]:
    print(1)
else:
    print(2)

1


In [53]:
if []:
    print(1)
else:
    print(2)

2


Такой код позволяет обезопасить себя от ошибок в коде. В данном случае, мы можем переписать всю конструкцию с `if - else` в виде одной удобной строки. Эта строка проверит полон ли респонса `meme_status` и ежели нет, то выдаст пустоту.

In [54]:
"Hello" if True else "Bye"

'Hello'

In [55]:
1 if ("a" in ['a', 'b']) or ('c' in [1, 4]) else 0

1

In [56]:
1 if ("a" in ['a', 'b']) and ('c' in [1, 4]) else 0

0

По аналогии можно вытащить всю остальную информацию со страницы.

Cоберем текстовое описание мема и его название.

In [57]:
def getText(soup):
    """
        Возвращает текстовые описания мема

        soup: объект bs4.BeautifulSoup
            суп текущей страницы

    """
    # название - идёт с самым большим заголовком h1, легко найти
    meme_name = soup.find('section', attrs={'class':'info'}).find('h1').text.strip()

    # достаём все тексты под картинкой
    body = soup.find('section', attrs={'class':'bodycopy'})

    # раздел about (если он есть), должен идти первым, берем его без уточнения класса
    meme_about = body.find('p')
    meme_about = "" if not meme_about else meme_about.text

    # раздел origin можно найти после заголовка Origin или History,
    # находим заголовок, определяем родителя и ищем следущего ребенка - наш раздел
    meme_origin = body.find(text='Origin') or body.find(text='History')
    meme_origin = "" if not meme_origin else meme_origin.parent.find_next().text

    # весь остальной текст (если он есть) можно запихнуть в одно текстовое поле
    if body.text:
        other_text = body.text.strip().split('\n')[4:]
        other_text = " ".join(other_text).strip()
    else:
        other_text = ""

    return {'Name': meme_name, 'About':meme_about, 'Origin':meme_origin, 'Other_text':other_text}

In [58]:
text_fields = getText(soup)

print("Название мема:\n{}\n\nО чем мем:\n{}\n\nПроисхождение:\n{}\n\nОстальной текст:\n{}...\n"\
      .format(text_fields['Name'], text_fields['About'], text_fields['Origin'], text_fields['Other_text'][:200]))

Название мема:
Doge

О чем мем:
Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog's face or captioned with interior monologues in Comic Sans font. The primary meme and iconography associated with Doge is the Shiba Inu named Kabosu, whose photos taken by her owner Atsuko Sato in early 2010 went viral across the internet, spawning numerous memes and larger trends in the following decades. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version as the memetic character continued to evolve.

Происхождение:
The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner's puppet show. In the episode titled "Biz Cas Fri 1"[2], Homestar calls Strong Bad his "d-o-g-e" while trying to distract him from his 

Наконец, создадим функцию, возвращающую всю информацию по текущему мему

In [59]:
def getMemeData(meme_page):
    """
        Запрашивает данные по странице, возвращает обработанный словарь с данными

        meme_page: string
            ссылка на страницу с мемом

    """

    # запрашиваем данные по ссылке
    response = requests.get(meme_page, headers={'User-Agent': UserAgent().chrome})

    if not response.ok:
        # если сервер нам отказал, вернем статус ошибки
        return response.status_code

    # получаем содержимое страницы и переводим в суп
    html = response.content
    soup = BeautifulSoup(html,'html.parser')

    # используя ранее написанные функции парсим информацию
    views = getStats(soup=soup, stats='views')
    videos = getStats(soup=soup, stats='videos')
    photos = getStats(soup=soup, stats='photos')
    comments = getStats(soup=soup, stats='comments')

    # дата
    date = soup.findAll('abbr', attrs={'class':'timeago'})[1].attrs['title']

    # текстовые поля
    text_fileds = getText(soup=soup)

    # составляем словарь, в котором будут хранится все полученные и обработанные данные
    data_row = {"date_added":date, "views":views,
                "videos":videos, "photos":photos, "comments":comments}
    data_row.update(text_fileds)

    return data_row

In [60]:
data_row = getMemeData('http://knowyourmeme.com/memes/doge')

In [61]:
data_row

{'date_added': '2013-07-24T16:29:55-04:00',
 'views': 14121050,
 'videos': 105,
 'photos': 1795,
 'comments': 923,
 'Name': 'Doge',
 'About': 'Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang term for "dog" that is primarily associated with pictures of Shiba Inus (nicknamed "Shibe") and internal monologue captions on Tumblr. These photos may be photoshopped to change the dog\'s face or captioned with interior monologues in Comic Sans font. The primary meme and iconography associated with Doge is the Shiba Inu named Kabosu, whose photos taken by her owner Atsuko Sato in early 2010 went viral across the internet, spawning numerous memes and larger trends in the following decades. Starting in 2017, Ironic Doge formats gained prevalence over the original wholesome version as the memetic character continued to evolve.',
 'Origin': 'The use of the misspelled word "doge" to refer to a dog dates back to June 24th, 2005, when it was mentioned in an episode of Homestar Runner\'s puppet show. In the ep

А теперь подготовим табличку, чтобы в неё записывать всё ~~награбленные~~ честно полученные данные, добавим в неё первую полученную строку и полюбуемся на результат

In [62]:
final_df = pd.DataFrame(columns=['Name',  'Type', 'Year', 'Origin',
                                 'date_added', 'views', 'videos', 'photos', 'comments',
                                 'About',  'Other_text'])

In [63]:
final_df = final_df.append(data_row, ignore_index=True).dropna(axis = 1)

In [64]:
final_df

,Name,Origin,date_added,views,videos,photos,comments,About,Other_text
0,Doge,"The use of the misspelled word ""doge"" to refer...",2013-07-24T16:29:55-04:00,14121050,105,1795,923,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer..."


Первый мем оказался в наших рукак. Еще раз убедимся что всё работает — пройдемся по списку из ссылок на мемы, полученных ранее в перменной `meme_links`.

In [65]:
from tqdm import tqdm_notebook

In [66]:
meme_links

['http://knowyourmeme.comhttps://knowyourmeme.com/memes/events/counter-strike-2-gun-sticker-controversy',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/bad-word-bathroom-challenge',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/woodrow-wilson-copypasta',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/people/pro-life-spiderman',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/everyone-knows-that-ulterior-motives-lost-media-song',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/noooo-put-me-back',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/esme-disillusion-st-mime-girl',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/i-fuck-so-much-pussy-i-might-switch-to-bussy',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/252-spa-choice-specs-beads-of-ruin-tera-fire-chi-yu',
 'http://knowyourmeme.comhttps://knowyourmeme.com/memes/events/sebastian-pinera-helicopter-crash',
 'http://knowyourmeme.comhttps://knowyourmeme.com/meme

In [67]:
for meme_link in tqdm_notebook(meme_links):
    try:
        data_row = getMemeData(meme_link)
        final_df = final_df.append(data_row, ignore_index=True)
        time.sleep(0.3)
    except:
        continue

  0%|          | 0/16 [00:00<?, ?it/s]

In [68]:
final_df = final_df.drop_duplicates().dropna(axis = 1)

In [69]:
final_df.shape

(1, 9)

In [70]:
final_df.head()

,Name,Origin,date_added,views,videos,photos,comments,About,Other_text
0,Doge,"The use of the misspelled word ""doge"" to refer...",2013-07-24T16:29:55-04:00,14121050,105,1795,923,Doge (pronounced /ˈdoʊdʒ/ DOHJ) is a slang ter...,"The use of the misspelled word ""doge"" to refer..."


Отлично! Всё работает, мемы качаются, данные наполняются и всё было бы хорошо, если бы не одно но — количество запросов, которое нам придётся сделать, чтобы всё получить.

# 2.  Момент истины

Вот он! Тот самый момент абсолютного триумфа, когда код дописан и всё, что нам, мирным собирателям, остаётся — запустить наш код на одну ночку. Кажется, что через страсть мы преобрели силу. Запускаем наш код по всем  страницам с мемами. На всякий случай обернём наш цикл в `try-except`. Мало ли что там с этими мемами бывает.

In [ ]:
# Немного красивых циклов. При желании пакет можно отключить и
# удалить команду tqdm_notebook из всех циклов
from tqdm import tqdm_notebook


final_df = pd.DataFrame(columns=['name', 'status', 'type', 'origin_year', 'origin_place',
                                 'date_added', 'views', 'videos', 'photos', 'comments',
                                 'tags', 'about', 'origin', 'other_text'])

for page_number in tqdm_notebook(range(1500), desc='Pages'):
    # собрали хрефы с текущей страницы
    meme_links = getPageLinks(page_number)
    for meme_link in tqdm_notebook(meme_links, desc='Memes', leave=False):
        # иногда с первого раза страничка не парсится
        for i in range(3):
            try:
                # пытаемся собрать по мему немного даты
                data_row = getMemeData(meme_link)
                # и закидываем её в таблицу
                final_df = final_df.append(data_row, ignore_index=True)
                # если всё получилось - выходим из внутреннего цикла
                break
            except:
                # Иначе, пробуем еще несколько раз, пока не закончатся попытки
                print('AHTUNG! parsing once again:', meme_link)
                continue
            time.sleep(0.3)

## 2.2 Тор - сын Одина

Иногда серверу надоедает общаться с одним и тем же человеком, делающим кучу запросов и сервер банит его. К сожалению, не только у людей запас терпения ограничен.

Приходится маскироваться. Для такой маскировки можно использовать разные способы, более того, один из них мы уже использовали, когда притворились человеком в нашем `request-header`. Для текущей же задачи, когда нас вероломно заблокировали по IP, нужно искать способы помощнее, чтобы иметь возможность этот IP менять. Конечно, как вариант можно было бы использовать прокси-сервера, тогда мы бы имели в запасе некоторое количество разных IP адресов, которые можно подставлять по мере "забанивания". Однако в этом подходе есть пара проблем: первая - нужно где-то раздобыть эти прокси, вторая - а что если ограниченного числа адресов нам не хватит и нужно больше?

В таком случае лучше всего нам подойдёт [Tor](https://www.torproject.org/). Вопреки пропагандируемому мнению, Tor используется не только преступниками, педофилами и прочими нехорошими террористами. Это, мягко говоря, далеко не так, и мы, мирные собиратели данных, являемся тому подтверждением. Всем прелестям, связанным с работой Tor, можно было бы посвятить несколько больших статеек, что собственно говоря уже и сделано. Подробнее про это можно почитатать по следующим ссылкам:

* [Как работает Tor](https://geektimes.ru/post/277578/)
* [Методы анонимности в сети](https://habrahabr.ru/post/204266/)
* [Прокси-сервер с помощью Tor](https://habrahabr.ru/company/etagi/blog/315002/)



In [ ]:
def checkIP():
    ip = requests.get('http://checkip.dyndns.org').content
    soup = BeautifulSoup(ip, 'html.parser')
    print(soup.find('body').text)

In [ ]:
checkIP()




## Почиташки

* [Неплохая инструкция](https://jarroba.com/anonymous-scraping-by-tor-network/) о самостоятельном парсинге через Tor без использования чужих готовых классов.
* [Репозиторий](https://github.com/DmitrySerg/memology) с исследованием мемов.
* [Оригинальный кодекс](http://starwars.wikia.com/wiki/Code_of_the_Sith) адепта тёмной стороны силы.